### Just a quick example of code that runs through all images in validation set and calculates competition metric

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# IPATH = '/content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls'
# !pip install {IPATH}/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links /content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls
# !pip install {IPATH}/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links /content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls
# !pip install   {IPATH}/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links /content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls
# !pip install {IPATH}/detectron2-0.5/detectron2 --no-index --find-links /content/drive/MyDrive/Kaggle/Sartorius/detectron2/whls 

In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/Kaggle/Sartorius')

In [ ]:
import torch
from detectron2 import model_zoo
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
import cv2
import os
import pycocotools.mask as mask_util
import numpy as np
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from typing import Any, Iterator, List, Union
import numpy as np
import detectron2
from detectron2.engine import DefaultTrainer
from types import SimpleNamespace
from detectron2.structures.masks import ROIMasks
from detectron2.layers.mask_ops import paste_masks_in_image
from detectron2.evaluation import DatasetEvaluators
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.evaluation import inference_on_dataset
from detectron2.checkpoint import DetectionCheckpointer
from scipy.ndimage.morphology import binary_fill_holes
from skimage.morphology import dilation, erosion
from skimage.measure import label
#from utils.post_processing import compute_overlaps_masks
from fvcore.transforms import HFlipTransform, NoOpTransform

# Post Processing Config

In [ ]:
class Config:

  THRESHOLDS = [.34646999695500497, .3134462119696301, .6716982906416183]#[0.2848161999265901,0.5901576776244156,0.6508687839015181]
  MIN_PIXELS = [60, 120, 60]#[60,80,50]
  # 该版本中以下都没用到
  PIXEL_CLASS_THRESHOLDS = [0.3, 0.25, 0.5]
  NMS_THRESH_TEST = 0.25

  anchor_generators_sizes = [[9], [17], [31], [64],[127]]
  anchor_generators_aspect_ratios = [[0.5, 1.0, 2.0]]

  pixel_mean = [128,128,128]
  pixel_std = [13.235,13.235,13.235]

# Custom TTA

需要custom一下tta的cnn类和transform 加入旋转

In [ ]:
class CustomizedRCNNWithTTA(GeneralizedRCNNWithTTA):
  def _reduce_pred_masks(self, outputs, tfms):
    # Should apply inverse transforms on masks.
    # We assume only resize & flip are used. pred_masks is a scale-invariant
    # representation, so we handle flip specially
    for output, tfm in zip(outputs, tfms):
        if any(isinstance(t, HFlipTransform) for t in tfm.transforms):
            output.pred_masks = output.pred_masks.flip(dims=[3])
    all_pred_masks = [np.transpose(o.pred_masks.cpu().numpy().squeeze(),(1,2,0)) for o in outputs]
    # voting
    overlaps = compute_overlaps_masks(all_pred_masks[0],all_pred_masks[1])

    for mm in range(overlaps.shape[0]):
        if np.max(overlaps[mm]) > 0.1:
            ind = np.argmax(overlaps[mm])
            all_pred_masks[0][:, :, mm] = (all_pred_masks[0][:, :, mm] + all_pred_masks[1][:, :, ind])/2
            #all_pred_masks[0][:, :, mm] = (mask > 0).astype(np.uint8)
            #result1['scores'][mm] = 0.5*(result1['scores'][mm]+result2['scores'][ind])
        else:
            all_pred_masks[0][:, :, mm] = 0
    all_pred_masks = [np.expand_dims(np.transpose(p,(2,0,1)),1) for p in all_pred_masks]
    return torch.tensor(all_pred_masks[0]).cuda()





# Utils

In [ ]:
# 安装了2.0.5的detectron后 以下代码可以获取每个像素是否为mask的概率 而不是0/1
# def paste_masks_in_image(masks, boxes, image_shape, threshold=0.5):
#     """
#     Copy pasted from detectron2.layers.mask_ops.paste_masks_in_image and deleted thresholding of the mask
#     """
#     assert masks.shape[-1] == masks.shape[-2], "Only square mask predictions are supported"
#     N = len(masks)
#     if N == 0:
#         return masks.new_empty((0,) + image_shape, dtype=torch.uint8)
#     if not isinstance(boxes, torch.Tensor):
#         boxes = boxes.tensor
#     device = boxes.device
#     assert len(boxes) == N, boxes.shape

#     img_h, img_w = image_shape

#     # The actual implementation split the input into chunks,
#     # and paste them chunk by chunk.
#     if device.type == "cpu":
#         # CPU is most efficient when they are pasted one by one with skip_empty=True
#         # so that it performs minimal number of operations.
#         num_chunks = N
#     else:
#         # GPU benefits from parallelism for larger chunks, but may have memory issue
#         num_chunks = int(np.ceil(N * img_h * img_w * BYTES_PER_FLOAT / GPU_MEM_LIMIT))
#         assert (
#             num_chunks <= N
#         ), "Default GPU_MEM_LIMIT in mask_ops.py is too small; try increasing it"
#     chunks = torch.chunk(torch.arange(N, device=device), num_chunks)

#     img_masks = torch.zeros(
#         N, img_h, img_w, device=device, dtype=torch.float32
#     )
#     for inds in chunks:
#         masks_chunk, spatial_inds = _do_paste_mask(
#             masks[inds, None, :, :], boxes[inds], img_h, img_w, skip_empty=device.type == "cpu"
#         )
#         img_masks[(inds,) + spatial_inds] = masks_chunk
#     return img_masks

# detectron2.layers.mask_ops.paste_masks_in_image.__code__ = paste_masks_in_image.__code__

# def BitMasks__init__(self, tensor: Union[torch.Tensor, np.ndarray]):
#   """
#   Args:
#   tensor: bool Tensor of N,H,W, representing N instances in the image.
#   """
#   device = tensor.device if isinstance(tensor, torch.Tensor) else torch.device("cpu")
#   tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device) # Original code: tensor = torch.as_tensor(tensor, dtype=torch.bool, device=device)
#   assert tensor.dim() == 3, tensor.size()
#   self.image_size = tensor.shape[1:]
#   self.tensor = tensor

# detectron2.structures.masks.BitMasks.__init__.__code__ = BitMasks__init__.__code__

In [ ]:
def precision_at(threshold, iou):
    matches = iou > threshold
    try:
      true_positives = np.sum(matches, axis=1) >= 1  # Correct objects
      false_positives = np.sum(matches, axis=1) == 0  # Extra objects
      false_negatives = np.sum(matches, axis=0) == 0  # Missed objects
      return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
    except:
      return 0,1,1


def score(pred, targ):
    enc_preds = post_process(pred)
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in enc_preds]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
        class_score[int(torch.mode(pred['instances'].pred_classes)[0])].append(p)
        #print("-"*50)
        #print(f"When threshold is {t}||Score is {p}||TP is {tp}||FP is {fp}||FN is {fn}")
    return np.mean(prec)

def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_
    rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return [x for x in runs]#' '.join(str(x) for x in runs)

def post_process(pred):
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    take = pred['instances'].scores >= Config.THRESHOLDS[pred_class]
    pred_masks = pred['instances'].pred_masks[take].cpu().numpy()

    res = []
    used = np.zeros(pred['instances'].pred_masks[0].shape)

    for idx,mask in enumerate(pred_masks):
        # 先填洞，筛选再去重
        #mask = binary_fill_holes(mask).astype(np.uint8)
        #mask = erosion(dilation(mask))

        # if mask.sum() >= Config.MIN_PIXELS[pred_class]: # skip predictions with small area
        #     #Delete overlaps
        #     used += mask
        #     mask[used > 1] = 0
        #     out_label = label(mask)
        #     # Remove all the pieces if there are more than one pieces
        #     if out_label.max() > 1:
        #         mask[()] = 0
        #     res.append(mask.astype(bool))

        # 去重后筛选
        mask = mask * (1-used)
        if mask.sum() >= Config.MIN_PIXELS[pred_class]: # skip predictions with small area
            used += mask
            res.append(mask.astype(bool))
    return res

# Generalized RCNN TTA

In [ ]:
class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:

                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
   
        return {"MaP IoU": np.mean(self.scores)}

In [ ]:
class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains pre-defined default logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can write your
    own training loop. You can use "tools/plain_train_net.py" as an example.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        # TTA+投票法 暂时不需要
        #model = CustomizedRCNNWithTTA(cfg, model)

        # 需要TTA时打开以下这行的注释
        #model = GeneralizedRCNNWithTTA(cfg, model)

        # 无TTA时 屏蔽上述两行用
        evaluator = [cls.build_evaluator(
            cfg, 'sartorius_val', # output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
        )]

        data_loader = cls.build_test_loader(cfg, 'sartorius_val')

        results_i = inference_on_dataset(model, data_loader, evaluator)
        return results_i

# Call

In [ ]:
def run(model_config,model_path,val_path,):
  cfg = get_cfg()
  cfg.INPUT.MASK_FORMAT='bitmask'
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
  cfg.MODEL.WEIGHTS = model_path
  cfg.TEST.DETECTIONS_PER_IMAGE = 1000
  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.OUTPUT_DIR = "output/"
  cfg.DATASETS.TEST = ('sartorius_val',)

  # TTA设置
  cfg.TEST.AUG.ENABLED = True
  cfg.TEST.AUG.MIN_SIZES = (1040,)
  cfg.TEST.AUG.FLIP = True
  #############################################################################
  # Cascade Team
  cfg.merge_from_file(model_zoo.get_config_file(model_config))
  cfg.MODEL.WEIGHTS = model_path #这个是用livecell迭代6503次的预训练权重
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.INPUT.MIN_SIZE_TEST = 1040
  cfg.INPUT.MAX_SIZE_TEST = 1408
  cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.35
  #cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
  #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
  #############################################################################


  cfg.freeze()
  
  DatasetCatalog.clear()
  MetadataCatalog.clear()

  register_coco_instances('sartorius_val',{},val_path, 
                          '/content/drive/MyDrive/Kaggle/Sartorius/input/sartorius-cell-instance-segmentation')

  val_ds = DatasetCatalog.get('sartorius_val')

  model = Trainer.build_model(cfg)

  DetectionCheckpointer(model, save_dir=model_path).resume_or_load(
      cfg.MODEL.WEIGHTS, resume=False
  )

  scr = Trainer.test_with_TTA(cfg, model)

  return scr["MaP IoU"]


In [ ]:
model_path = "/content/drive/MyDrive/Kaggle/Sartorius/model/TEAM/cascade_cv316/1218_cv316.pth"#"/content/drive/MyDrive/Kaggle/Sartorius/model/finetuned/cascade_v0/fold_1/model_best.pth"#'/content/drive/MyDrive/Kaggle/Sartorius/model/finetuned/cascade_v0/fold_2/model_best.pth' 
model_config = "Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml" ##"Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
val_path = "/content/drive/MyDrive/Kaggle/Sartorius/input/all/annotations_val.json"#/content/drive/MyDrive/Kaggle/Sartorius/input/fold/coco_cell_valid_fold1.json"

In [ ]:
class_score = {0:[],1:[],2:[]} #shy5sy, astro, cort
run(model_config,model_path,val_path)

In [ ]:
# 显示不同种类细胞的mAP
for i in range(3):
  print(f"Class {i} is ",np.mean(class_score[i]))

Class 0 is  0.22050477051156162
Class 1 is  0.17676962157585002
Class 2 is  0.3932212240991607


# OPTUNA

调参用

In [ ]:
!pip install optuna

In [ ]:
import optuna
from optuna.samplers import TPESampler
# parameters search
class params_optim():
  def __init__(self,train_time):
    self.train_time = train_time
  
  def search(self):

    study = optuna.create_study(study_name="Post_Processing_Parameters",direction="maximize",sampler=TPESampler())
    study.optimize(self.objective,n_trials=self.train_time)
    best_trial = study.trial
    print(best_trial)
    return best_trial.params

  def objective(self,trial):
    params = {"threshold_class_1":trial.suggest_uniform("threshold_class_1",0.1,0.5),
              "threshold_class_2":trial.suggest_uniform("threshold_class_2",0.3,0.7),
              "threshold_class_3":trial.suggest_uniform("threshold_class_3",0.4,0.8),

              "min_pixel_class_1":trial.suggest_int("min_pixel_class_1",30,100,10),
              "min_pixel_class_2":trial.suggest_int("min_pixel_class_2",40,200,20),
              "min_pixel_class_3":trial.suggest_int("min_pixel_class_3",30,100,10),
              }

    Config.MIN_PIXELS = [params["min_pixel_class_1"], params["min_pixel_class_2"], params["min_pixel_class_3"]] # shsy5y, astro, cort

    scr = run(model_config,model_path,val_path)

    return scr
  

In [ ]:
# 进行120次实验
params = params_optim(120).search()